# Generate Data set 

In [3]:
!python ../main_generate_dataset.py


Bad key "text.kerning_factor" on line 4 in
/Users/seongjungkim/anaconda3/envs/python3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution
100%|████████████████████████████████████| 11000/11000 [01:09<00:00, 158.50it/s]
consume_time : 85.67403197288513
transaction units : 128.39363044663727
Finish generate detection dataset!


# Training Model

In [ ]:
!python ../main_training.py

2021-07-15 15:13:07.466387: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2021-07-15 15:13:07.466641: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.
